# Feature selection

In this notebook we will look again at the brain structure volumes of preterm babies. We will see how feature selection can prevent overfitting, improve performance of models and how it can also help in feature interpretation.

In [ ]:
###################################
## RUN THIS
###################################
# this code is to suppress warnings
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=ConvergenceWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
####################################

## Brain structure volumes

We will load again the dataset of 86 brain structure volumes of 164 preterm babies. We will revisit prediction of GA from the volumes. Recall, that Multivariate Linear Regression resulted in overfitting of the data and Lasso and Ridge penalties were successful in reducing the overfitting. 

### Load data

The code below loads the data, creates the feature matrix and the target vector and performs the feature scaling.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

# read spreadsheet using pandas
data = pd.read_csv("datasets/GA-structure-volumes-preterm.csv",header=None)
# convert from 'DataFrame' to numpy array
structure_volumes = data.to_numpy()
# Features
X = structure_volumes[:,1:]
# Targets
y = structure_volumes[:,0]
# checking the size of the feature and target arrays 
# note they must agree in the first dimension
print('Features shape: {}; Targets shape: {}'.format(X.shape,y.shape)) 
# we have 86 features and 164 samples

# Scale features
X = StandardScaler().fit_transform(X)
print('Performed feature scaling.')

We are interested in finding out which features are selected by our models as most predictive of the age at scan. The code below reads the structure names and stores them as a `dataframe` object `structure_names`.

In [ ]:
# read file with structure names
structure_names = pd.read_csv('datasets/labels', header = None, sep='\t')
structure_names[1]

### Multivariate linear regression 

Multivariate linear regression overfitted the data. Rembember the performance of linear regression to see whether feature selection can help us improve it.

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score

model = LinearRegression()
scores = cross_val_score(model, X, y, scoring = 'neg_mean_squared_error')
print('Linear regression: Cross-validated RMSE is ', round(np.sqrt(-scores.mean()),2))

### Ridge

Previously, we have found that optimal `alpha` for Ridge regression was around 45. This setting significanlty reduced overfitting. Let's rerun this model. Remember the performance of Ridge regression as a baseline for good performance.

In [ ]:
from sklearn.linear_model import Ridge

model = Ridge(alpha = 45)
scores = cross_val_score(model, X, y, scoring = 'neg_mean_squared_error')
print('Ridge regression: Cross-validated RMSE is ', round(np.sqrt(-scores.mean()),2))

We will now explore different feature selection techniques in Scikit-learn.


## Univatiate feature selection

### Pearson's correlation coefficient

Correlation coefficient can be calaculated using function `pearsonr` from `scipy.stats` module. High correlation (positive and negative) means close to linear relationship with target values. Note that majority, but not all, volumes are highly correlated with GA.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

from scipy.stats import pearsonr

n = X.shape[1]
cc = np.zeros(n)
for i in range(n):
    cc[i]=pearsonr(X[:,i],y)[0]

plt.figure(figsize = [16,4])
plt.bar(np.arange(n),cc)
plt.title('Pearsons correlation coefficient', fontsize = 18)
plt.xlabel('Feature', fontsize = 16)
plt.ylabel('Correlation coefficient', fontsize = 16)
plt.axis([-1,86,0,1])

### F-score

Scikit-learn works with F-values rather than Pearson's Correlation Coefficient. However these two are equivalent for feature selection.

The F-value can be calculated using `f_regression` in `sklearn`. 

**Activity 1.1:** Complete the code below to plot F-scores using a `bar` plot.

In [ ]:
from sklearn.feature_selection import f_regression

f_score = f_regression(X,y)[0]

# plot f-scores
plt.figure(figsize = [16,4])
plt.bar(np.arange(n),None)
plt.title('F-value', fontsize = 18)
plt.xlabel('Feature', fontsize = 16)
plt.ylabel('F-value', fontsize = 16)

**Activity 1.2:** Plot relationship between Person's correlation coefficient and F-score using `plot`. Are they equivalent?

In [ ]:
# plot relationship
plt.plot(None,None,'*')
plt.xlabel("Person's correlation coefficient", fontsize = 16)
plt.ylabel('F-value', fontsize = 16)

### Selecting features based on F-value
<img src="pictures/brain.png" width = "250" style="float: right;"> 

We can select top scoring features to improve performance of the method. Scikit-learn offers two in-build functions to do that: `SelectKBest` and `SelectPercentile`. Let's start by selecting 4 best features using `SelectKBest`.

Run the code below to transform the feature matrix `X` into the matrix of selected features `X_selected`. 

**Activity 1.3:** Check
* the size of the new matrix - is it what you expect?
* the indices of the features that have been selected.
* the names of the selected features.

In [ ]:
from sklearn.feature_selection import SelectKBest

# define feature selection model
k=4
selector = SelectKBest(f_regression, k = k)

# select features
X_selected = selector.fit_transform(X,y)

# Shape of the matrix
print('Shape of the new matrix: ', X_selected.shape)

# Indices of the selected features
ind = np.where(selector.get_support())[0]
print('Indices: ', ind)

# Print the names of the selected structures
print('\n')
for i in range(k):
    print(structure_names.loc[ind[i],1])

### Univariate feature selection for improved prediction

**Activity 1.4:** Let's now apply the multivariate linear regression to the selected features and see whether performance improves. 

In [ ]:
# Select and fit linear regression model to selected features
model = None
model.fit(None,y)

# Calculate and print RMSE
scores = cross_val_score(model, X_selected, y, scoring = 'neg_mean_squared_error')
print('Linear regression: Cross-validated RMSE is ', round(np.sqrt(-scores.mean()),2))

We see that we reduced overfitting but we did not reach performance of Lasso or Ridge. 

**Activity 5:** Try to vary the number of selected features to see what works best. Remember this performance as the best for univariate feature selection.

## Exercise 1

Select 4 top scoring features using mutual information. Do you obtain the same or different features as for correlation coefficient?

In [ ]:
from sklearn.feature_selection import mutual_info_regression

# set number of features to select
k=4

# Create feature selector
selector = None

# select features
X_selected = None

# Indices of the selected features
ind = None

# Print the names of the selected structures
print('\n')
for i in range(k):
    print(structure_names.loc[ind[i],1])

## Model based feature selection

### Lasso

We will now select the features based on `Lasso` model. We have previously found that setting `alpha=0.16` results in a best Lasso model for our example. Code below creates the model, calculates its performance and prints out the number of sparse coefficients.

In [ ]:
from sklearn.linear_model import Lasso

model = Lasso(alpha=0.16)
scores = cross_val_score(model, X, y, scoring = 'neg_mean_squared_error')
print('Lasso regression: Cross-validated RMSE is ', round(np.sqrt(-scores.mean()),2))

model.fit(X,y)
print('\n Non-zero coefficients')
print(model.sparse_coef_)
print('\n There are {} non-zero coefficients.'.format(model.sparse_coef_.count_nonzero()))

The code below extracts indices of non-zero Lasso coefficients and lists the names of the selected structures.

In [ ]:
# indices of non-zero elements
ind = model.sparse_coef_.nonzero()[1]
print('Indices of non-zero elements: ', ind)
print('\n')

# print names of selected structures
print('Selected structures: \n')
for i in range(ind.size):
    print(structure_names.loc[ind[i],1])

## Exercise 2

We will now use `LassoCV` model for feature selection but fit the selected features using `LinearRegression`. 
* Implement the feature selection using selector `SelectFromModel` with `LassoCV` model.
* Calculate the performance of `LinearRegression` using the selected features.
* Experiment with different thresholds for Lasso coeficients to see which one results in best performance.

In [ ]:
from sklearn.linear_model import LassoCV
from sklearn.feature_selection import SelectFromModel

# Create selector with LassoCV model
selector = None

# Perform feature transformation
X_selected = None

# Create and fit linear regression model to selected features
model = None
model.fit(None,y)

# Calculate and print RMSE
scores = cross_val_score(model, None, y, scoring = 'neg_mean_squared_error')
print('Linear regression: Cross-validated RMSE is ', round(np.sqrt(-scores.mean()),2))

# List the number and names of the selected features
ind = None
print('\nSelected {} features: '.format(ind.size),)
for i in range(ind.size):
    print(structure_names.loc[ind[i],1])

We see that feature selection using LassoCV with optimised threshold results in performance of Linear regression similar to optimised Ridge. This is better than for univariate feature selection.

### Random forest

Random forest is very resilient to overfitting. Let's now fit the random forest regressor to the data and see how it performs. Note, that Random forest allows for non-linear models and it is therefore not surprising that it outperforms Linear Regression Models.

In [ ]:
from sklearn.ensemble import RandomForestRegressor
 
# Select and fit the model
model = RandomForestRegressor(n_estimators=20)

# Calculate CV RMSE
scores = cross_val_score(model, X, y, scoring = 'neg_mean_squared_error')
print('Cross-validated RMSE is ', round(np.sqrt(-scores.mean()),2))

**Activity 2.1:** Feature importances can be access as `model.feature_importances_`. Plot the feature importances using a `bar` plot.

In [ ]:
# fit the model
model.fit(X,y)

# plot feature importances
plt.figure(figsize = [16,4])
n = X.shape[1]
plt.bar(np.arange(n),None)
plt.title('Feature importances', fontsize = 18)
plt.xlabel('Features', fontsize = 16)
plt.ylabel('importances', fontsize = 16)

**Activity 2.2:** Use selector `SelectFromModel` to select the features from `RandomForestRegressor(n_estimators=20)`. Choose threshold 0.05 and print the names of the selected features. Are they consistent with the ones selected by Lasso or Correlation Coefficient?

In [ ]:
# Create selector with LassoCV model
selector = None

# Perform feature transformation
X_selected = selector.fit_transform(X, y)

# List the number and names of the selected features
ind = selector.get_support(indices=True)
print('Selected {} features: '.format(ind.size),)
for i in range(ind.size):
    print(structure_names.loc[ind[i],1])

## Recursive feature elimination

Scikit learn offers functions `RFE` and `RFECV` to perform recursive feature elimination. Any model can be used to do that, and this time we will chose `Ridge` regression.  

Let's first find 6 best features using `RFE` with `Ridge`. Run the code below to fit `RFE` model and print the names of the selected features.

In [ ]:
from sklearn.feature_selection import RFE

k=6

# create ranking model
model = Ridge(alpha=45)

# create selector
selector = RFE(model, n_features_to_select=k)

# fit selector
selector.fit(X,y)

# Print the indices of the selected features
ind = np.where(selector.get_support())[0]
print('Indices: ', ind)

# Print the names of the selected structures
print('\n')
for i in range(k):
    print(structure_names.loc[ind[i],1])

**Activity 3.1:** Transform the features, fit linear regression and calculate CV RMSE to see whether we reduced overfitting.

In [ ]:
# Select features
X_selected = None

# Linear regression
model = None
scores = cross_val_score(model, X_selected, y,scoring = 'neg_mean_squared_error')
print('Linear regression: Cross-validated RMSE is ', round(np.sqrt(-scores.mean()),2))

## Exercise 3
Let's now use method `RFECV` that can also automatically select optimal number of features using cross-validation. Write the code to:
* Fit the `RFECV` feature selection with `Ridge(alpha=45)` ranking model
* Transform the features and fit the `Ridge(alpha = 45)` model to the selected features
* Calculate the CV RMSE
* Print indices of selected features
* Print number of selected features

In [ ]:
from sklearn.feature_selection import RFECV

# create ranking model
model = None

# Create selector
selector = None

# Fit the selector and transform the features
X_selected = None

# Calculate performace of Ridge with selected features
scores = None
print('Linear regression: Cross-validated RMSE is ', round(np.sqrt(-scores.mean()),2))

# Print indices of the selected features
ind = None
print('Indices: ', ind)

# Print number of selected features
print('Number of selected features: ', ind.size)

### Recursive feature elimination using Random Forest

**Activity 3.2:** Perform recursive feature elimination using `RFECV` and `RandomForestRegressor(n_estimators=20)`. Be patient, this process might take time.

In [ ]:
from sklearn.feature_selection import RFECV

model = RandomForestRegressor(n_estimators=20)
selector = None
selector.fit(X,y)

# Print selected features
ind = np.where(selector.get_support())[0]
print('Indices: ', ind)

print('Number of selected features: ', ind.size)

**Activity 3.3:** Transform the features and fit the `RandomForestRegressor(n_estimators=20)` to see whether CV RMSE improved.

In [ ]:
# Select features
X_selected = None

# Random Forest with reduced features
model = None
scores = cross_val_score(model, X_selected, y, scoring = 'neg_mean_squared_error')
print('Linear regression: Cross-validated RMSE is ', round(np.sqrt(-scores.mean()),2))

# Conclusion

We have seen that feature selection can prevent overfiting and improve performance of the model. We have also seen that Random forest is very resilient against overfitting and our it example did not benefit from feature selection. On contrary, it is a very good tool for selecting features for other methods.

We have also seen that selected features varied a lot dependent on the selection method. We therefore need to be careful when interpreting the selected features.